EJERCICIO PUENTES - PABLO MARTINEZ

In [78]:
cd

C:\Users\Tine


In [79]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *

Utilizamos la clase problem de la que heredara la clase que resuelve el ejercicio

In [80]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Dado un estado, devuelve las posibles acciones de ese estado ?¿?¿?¿? no se si es asi 100%"""
        """donde vamos a tener las acciones"""
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """dado una accion y un estado ejecuta l accion y devuelve el estado siguiente"""
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Tenemos que tener e cuenta que vamos a pasar como atributos:

    una lista con los tiempos de cada persona (n)
    una lista con:
        1er elemento el tiempo de la linterna
        n elementos booleanos con las posiciones de las personas (False: izquierda  True:derecha)
        1 elemento booleano con la posicion de la linterna (False: izquierda  True:derecha)
    el goal, que es una lista con n elementos booleanos a True (que es el lado objetivo)

In [81]:
class Puente(Problem):
    
    personas = list()
    
    def __init__(self,personas,initial,goal):
        self.personas = personas
        Problem.__init__(self,initial,goal)
        
    #funcion que devuelve una tupla con las personas que estan en el lado de la linterna
    #la necesitamos para devolver todas las posibles acciones de un estado (depende de la linterna)
    def devuelvePersonas(self,state):
        size = len(list(state))
        lista = list()
        posLinterna = state[size-1]
        
        for i in range(size-2):#porque el primero es el tiempo y el ultimo la pos de la linterna
            if(state[i+1] == posLinterna):
                lista.append(i+1)
        
        return tuple(lista)
        
    def actions(self, state):
        accs=list()
        tupla = tuple(self.devuelvePersonas(state))
        
        if(state[0] > 0):#si el tiempo de la linterna se ha consumido
            for i in tupla:
                for j in tupla:
                    accs.append((i,j))#todas las posibles combinaciones de las personas que estan en el lado de la linterna
            
        return tuple(accs)#lo devolvemos en formato tupla para que no sea mofificable
    
    def result(self,state,action):
        listaA = list(action) #parseamos a lista la accion porque es una tupla (persona1,persona2)
        listaE = list(state)#parseamos a lista porque el estado contiene varios argumentos (tiempo,personas,linterna)
        
        persona1 = self.personas[listaA[0]-1]#restamos uno porque en las acciones el primer valor de persona es 1 y no 0
        persona2 = self.personas[listaA[1]-1]#el contenido de la accion representa las posiciones en la lista personas
        
        listaE[0] -= max(persona1,persona2)#restamos al tiempo de la linterna el máximo tiempo de las dos personas
        
        listaE[len(listaE)-1] = not listaE[len(listaE)-1] #cambiamos la posicion de la linterna
        listaE[listaA[1]] = listaE[len(listaE)-1]#añadimos a las dos persnas al otro lado del puente
        listaE[listaA[0]] = listaE[len(listaE)-1]#asignamos el valor de la linterna para cambiar de lado
        
        self.state = list(listaE)#guardamos el estado resultante
        
        return list(listaE)
        
    def goal_test(self,state):
        result = True
        for i in range(len(state) -1):#comprobamos que todas las personas y la linterna esten en el lado derecho(True)
            if(state[i+1] == False):
                result = False
            
        return result
        

In [85]:
puente1 = Puente((5,10),(50,False,False,False),(True,True))

In [86]:
puente1.actions(puente1.initial)

((1, 1), (1, 2), (2, 1), (2, 2))

In [87]:
puente1.goal_test((50,True,True,True))

True

In [88]:
puente2 = Puente((5,10,15),(50,False,False,False,False),(True,True,True))

In [89]:
puente3 = Puente((5,10,15,20),(60,False,False,False,False,False),(True,True,True,True))

In [90]:
puenteFinal = Puente((10,30,60,80,120),(300,False,False,False,False,False,False),(True,True,True,True,True))

Los graph (con control de repetidos) no me deja ejecutarlos, me pone: unhashable type: 'list' (tipo no manipulable)

Los graph no tiene sentido utilizarlos porque tenemos atributos tipo tuple que van a impedir la modificacion de nodos, por eso solo utilizamoos tree

In [95]:
%%timeit 1
breadth_first_tree_search(puente1).solution()

45.3 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [96]:
%%timeit 1
depth_first_tree_search(puente1).solution()

113 µs ± 4.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [97]:
%%timeit 1
breadth_first_tree_search(puente2).solution()

1.54 ms ± 71.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [98]:
%%timeit 1
depth_first_tree_search(puente2).solution()

391 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [99]:
%%timeit 1
breadth_first_tree_search(puente3).solution()

105 ms ± 9.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
%%timeit 1
depth_first_tree_search(puente3).solution()

1.29 ms ± 57.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [101]:
%%timeit 1
breadth_first_tree_search(puenteFinal).solution()

6.38 s ± 74.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
%%timeit 1
depth_first_tree_search(puenteFinal).solution()

27.9 ms ± 854 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Podemos ver que en este problema, los algoritmos en profundidad son mucho más rápidos que los de anchura.

In [74]:
#breadth_first_graph_search(puente1).solution()

In [75]:
#breadth_first_graph_search(puente2).solution()

In [76]:
#breadth_first_graph_search(puente3).solution()

In [ ]:
#breadth_first_graph_search(puenteFinal).solution()

In [77]:
#depth_first_graph_search(puente1).solution()

In [ ]:
#depth_first_graph_search(puente2).solution()

In [ ]:
#depth_first_graph_search(puente3).solution()

In [ ]:
#depth_first_graph_search(puenteFinal).solution()